# TuneWeaver: Data Collection and Preparation for Playlist Recommendation Analysis

**Project:** TuneWeaver - Enhanced A/B/n Testing & Production-Ready Pipeline for Serendipity Playlists
**Notebook Purpose:** This Jupyter Notebook handles the initial and crucial phase of the TuneWeaver project: **Data Collection and Preparation**. Its primary goal is to acquire a substantial dataset of music tracks from the Spotify API, including their metadata and audio features. Subsequently, it performs thorough pre-processing and cleaning to prepare this data for the downstream tasks of algorithm development, simulation, and A/B/n testing analysis.

**Key Activities in this Notebook:**
1.  **Spotify API Interaction:** Utilizing the Spotipy library to connect to the Spotify Web API.
2.  **Data Acquisition:** Fetching data for over 5,000 tracks across a diverse set of genres. This includes:
    * Track metadata (ID, name, artist, album, popularity).
    * Audio features (e.g., danceability, energy, valence, tempo).
3.  **Initial Data Storage:** Saving the raw, collected data into a `songs.csv` file.
4.  **Detailed Pre-processing:**
    * Loading and inspecting the raw dataset.
    * Comprehensive missing value analysis and imputation.
    * Normalization of numerical features to ensure consistent scaling for subsequent analysis and modeling.
5.  **Final Processed Data Storage:** Saving the cleaned and transformed dataset as `songs_processed.csv`.

**Outcome:** A well-structured, clean, and normalized dataset (`songs_processed.csv`) ready for the next stages of the TuneWeaver project, particularly for implementing and evaluating different playlist recommendation algorithms.

**Professional Practices Emphasized:**
* Secure handling of API credentials (guidance provided).
* Clear documentation and comments.
* Systematic data cleaning and transformation steps.
* Reproducibility considerations.

## Part 1: Data Collection using Spotify API (Spotipy)

This section outlines the process of collecting music track data. We will use the [Spotipy](https://spotipy.readthedocs.io/) library, a lightweight Python client for the Spotify Web API.

**Goal:** To compile a dataset of at least 5,000 unique songs, including their audio features and metadata, covering a variety of genres.

In [1]:
# Cell 3: Imports and Configuration for Data Collection

# Core libraries
import pandas as pd
import time
import os # For securely accessing environment variables

# Spotify API client
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

# --- Spotify API Credentials Configuration ---
# BEST PRACTICE: Store your API credentials as environment variables
# and load them using os.getenv(). Avoid hardcoding them directly in your script/notebook.
#
# Example of setting environment variables (in your terminal/shell before launching Jupyter):
# export SPOTIPY_CLIENT_ID='your_actual_client_id'
# export SPOTIPY_CLIENT_SECRET='your_actual_client_secret'
#
# Then, you can load them like this:
# client_id = os.getenv('SPOTIPY_CLIENT_ID')
# client_secret = os.getenv('SPOTIPY_CLIENT_SECRET')
#
# If you don't have them set as environment variables, you can temporarily
# assign them below FOR DEVELOPMENT PURPOSES ONLY.
# **REMEMBER TO REMOVE OR REPLACE WITH os.getenv() BEFORE COMMITTING TO GITHUB.**

client_id = os.getenv('SPOTIPY_CLIENT_ID')
client_secret = os.getenv('SPOTIPY_CLIENT_SECRET')

# Fallback for development if environment variables are not set (NOT FOR PRODUCTION/GITHUB)
# If you uncomment the lines below to hardcode, replace 'YOUR_CLIENT_ID' and 'YOUR_CLIENT_SECRET'
if not client_id or not client_secret:
    print("⚠️ Spotify API credentials not found in environment variables.")
    print("   Please set SPOTIPY_CLIENT_ID and SPOTIPY_CLIENT_SECRET environment variables for secure credential management.")
    print("   For temporary local development only, you can uncomment and fill the lines below.")
    print("   Ensure these are NOT committed to version control if hardcoded.")
    # client_id = 'YOUR_CLIENT_ID'  # <<< TEMPORARY: REPLACE WITH YOUR CLIENT ID
    # client_secret = 'YOUR_CLIENT_SECRET' # <<< TEMPORARY: REPLACE WITH YOUR CLIENT SECRET

# --- Data Collection Parameters ---
# Filename for the raw collected data from Spotify
raw_output_filename = 'songs.csv'

# Target number of unique tracks to collect.
# The project plan specifies 5,000+ tracks.
total_tracks_target = 5500 # Aiming slightly above 5000 for a buffer

# Number of tracks to attempt to fetch per genre.
# Spotify's `recommendations` endpoint limit is 100 per call.
# This parameter, combined with the number of genres, helps reach `total_tracks_target`.
num_tracks_per_genre = 400 # Adjusted to help ensure target is met with genre diversity

# Define a diverse list of genres to fetch tracks from.
# These serve as seeds for Spotify's recommendation engine.
genres = [
    "pop", "rock", "hip-hop", "electronic", "classical", "jazz", "r-n-b",
    "country", "folk", "metal", "blues", "funk", "soul", "indie",
    "latin", "reggae", "punk", "alternative", "dance", "ambient"
]

# --- Initial Check and Confirmation ---
# This check is crucial to ensure credentials are set before proceeding.
if not client_id or client_id == 'YOUR_CLIENT_ID' or \
   not client_secret or client_secret == 'YOUR_CLIENT_SECRET':
    raise ValueError("Spotify API credentials not set. Halting execution.")
else:
    print("✅ Configuration loaded successfully.")
    print(f"   Targeting approximately {total_tracks_target} unique tracks.")
    print(f"   Raw data will be saved to: '{raw_output_filename}'")
    print(f"   Attempting to fetch up to {num_tracks_per_genre} tracks per genre from {len(genres)} genres.")
    print("   Note: Actual Spotify API credentials are not displayed for security.")

✅ Configuration loaded successfully.
   Targeting approximately 5500 unique tracks.
   Raw data will be saved to: 'songs.csv'
   Attempting to fetch up to 400 tracks per genre from 20 genres.
   Note: Actual Spotify API credentials are not displayed for security.


In [5]:
# New Cell - Test API Call with 'requests' Library Directly

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import requests # For making direct HTTP requests
import json     # For pretty printing the JSON response

print("--- Testing Direct API Call with 'requests' Library ---")

# Assuming client_id and client_secret are still in memory from your original Cell 3
if 'client_id' in locals() and 'client_secret' in locals() and \
   client_id != 'YOUR_CLIENT_ID' and client_secret != 'YOUR_CLIENT_SECRET':
    
    access_token = None
    try:
        # 1. Get an access token using Spotipy's auth manager
        print("Attempting to get an access token using Spotipy auth...")
        auth_manager_direct_test = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
        token_info = auth_manager_direct_test.get_access_token(as_dict=False) # Get as string
        
        if token_info:
            access_token = token_info
            print("✅ Successfully obtained access token.")
            # print(f"   Access Token (first 10 chars): {access_token[:10]}...") # Uncomment to verify token format
        else:
            print("❌ Failed to obtain access token using Spotipy auth manager.")
            raise Exception("Failed to get access token")

        # 2. Define the API endpoint URL and headers for a direct request
        # We'll try fetching the same Miles Davis artist info
        artist_id_to_test = '0kbYTNQb4Pb1rPbbaX0p5l'
        # This is the standard Spotify API endpoint for an artist
        spotify_api_url = f"https://api.spotify.com/v1/recommendations" 
                                   
        headers = {
            'Authorization': f'Bearer {access_token}'
        }

        print(f"\nMaking direct GET request to: {spotify_api_url}")
        print(f"Using Authorization Header: Bearer {access_token[:10]}...{access_token[-5:]}")

        # 3. Make the GET request using the 'requests' library
        response = requests.get(spotify_api_url, headers=headers)

        print(f"\n--- Direct Request Response ---")
        print(f"Status Code: {response.status_code}")
        
        # Try to print the response content (usually JSON)
        try:
            response_content = response.json()
            print("Response JSON:")
            # Pretty print the JSON
            print(json.dumps(response_content, indent=2)) 
            
            if response.status_code == 200 and 'name' in response_content:
                print(f"\n✅ Successfully fetched artist '{response_content['name']}' via direct requests call!")
            elif response.status_code == 404:
                print(f"\n❌ Received HTTP 404 (Not Found) from direct requests call as well.")
                print(f"   This suggests the issue might be with reaching the Spotify API URL correctly,")
                print(f"   or a fundamental problem with the API endpoint itself, or network interference.")
            else:
                print(f"\n⚠️ Received status {response.status_code}. Expected 200 for success.")

        except requests.exceptions.JSONDecodeError:
            print("Response Content (not valid JSON):")
            print(response.text)
            if response.status_code == 404:
                 print(f"\n❌ Received HTTP 404 (Not Found) from direct requests call (response not JSON).")
            
    except spotipy.SpotifyException as e: # For issues getting the token via Spotipy
        print(f"❌ Spotify (Spotipy) error during token acquisition: HTTP {e.http_status} - {e.msg}")
        print(f"   Details: {e}")
    except requests.exceptions.RequestException as e: # For errors during the requests.get() call itself (e.g., network error)
        print(f"❌ 'requests' library error during direct API call: {type(e).__name__} - {e}")
        print(f"   This could indicate a network issue or problem reaching the Spotify domain.")
    except Exception as e:
        print(f"❌ An unexpected error occurred during the direct API call test: {type(e).__name__} - {e}")
else:
    print("❌ Spotify client_id or client_secret are not properly set up from your original Cell 3.")
    print("   Cannot run the direct API call test without valid authentication to get a token.")

print("\n--- Direct API Call Test Complete ---")

--- Testing Direct API Call with 'requests' Library ---
Attempting to get an access token using Spotipy auth...
✅ Successfully obtained access token.

Making direct GET request to: https://api.spotify.com/v1/recommendations
Using Authorization Header: Bearer BQDGwTcRMN...nVB6k

--- Direct Request Response ---
Status Code: 404
Response Content (not valid JSON):


❌ Received HTTP 404 (Not Found) from direct requests call (response not JSON).

--- Direct API Call Test Complete ---


In [3]:
# Cell 4: Define the Spotify Data Fetching Function

def fetch_spotify_track_data(client_id_param: str,
                               client_secret_param: str,
                               genres_list: list,
                               num_tracks_per_genre_target: int,
                               total_tracks_overall_target: int) -> pd.DataFrame | None:
    """
    Fetches track metadata and audio features from the Spotify API for a list of seed genres.

    Args:
        client_id_param (str): Spotify API Client ID.
        client_secret_param (str): Spotify API Client Secret.
        genres_list (list): A list of genre strings to use as seeds for recommendations.
        num_tracks_per_genre_target (int): The desired number of tracks to fetch per genre.
                                           The function will attempt to get up to this many new tracks
                                           for each genre, subject to API limits and availability.
        total_tracks_overall_target (int): The minimum total number of unique tracks to collect across all genres.
                                           The function will stop fetching new genres once this target is met or exceeded.

    Returns:
        pd.DataFrame | None: A pandas DataFrame containing the collected track data (including metadata
                              and audio features like 'danceability', 'energy', 'valence', etc.),
                              or None if a critical error occurs during authentication or if no data
                              can be fetched. The DataFrame includes a 'genre_seed' column indicating
                              the genre that led to the track's discovery.
    """
    if not client_id_param or client_id_param == 'YOUR_CLIENT_ID' or \
       not client_secret_param or client_secret_param == 'YOUR_CLIENT_SECRET':
        print("❌ Error: Spotify Client ID or Secret not properly provided to `Workspace_spotify_track_data` function.")
        print("   Please ensure Cell 3 is correctly configured with valid credentials.")
        return None

    try:
        # Authenticate with Spotify API using Client Credentials Flow
        # This flow is suitable for server-to-server authentication where no user is directly involved.
        auth_manager = SpotifyClientCredentials(client_id=client_id_param, client_secret=client_secret_param)
        sp = spotipy.Spotify(auth_manager=auth_manager)
        print("✅ Successfully authenticated with Spotify API for data fetching.")
    except Exception as e:
        print(f"❌ CRITICAL ERROR during Spotify authentication in `fetch_spotify_track_data`: {e}")
        print("   Please double-check your Client ID and Secret in Cell 3 and your network connection.")
        return None

    all_tracks_data = []  # List to store dictionaries, each representing a track's complete data
    track_ids_seen = set()  # Set to keep track of collected track IDs to ensure uniqueness across all API calls

    print(f"\n🔄 Initializing data collection. Overall target: {total_tracks_overall_target} unique tracks.")
    print(f"   Will attempt to fetch up to {num_tracks_per_genre_target} tracks per genre from {len(genres_list)} seed genres.")

    for genre_item in genres_list:
        # Check if the overall target for unique tracks has been met before processing a new genre
        if len(track_ids_seen) >= total_tracks_overall_target:
            print(f"\n🏁 Overall track target ({total_tracks_overall_target}) met or exceeded. Halting further genre processing.")
            break

        print(f"\n🎵 Processing Genre: '{genre_item}' (Collected so far: {len(track_ids_seen)} unique tracks)")
        current_genre_track_count = 0 # Counter for tracks collected for the current genre
        
        # Spotify's `recommendations` endpoint has a limit of 100 tracks per API call.
        # We'll use a slightly lower limit per call (e.g., 75) to be cautious and allow for multiple calls if needed.
        limit_per_api_call = 75 

        # Calculate the number of API calls potentially needed to reach `num_tracks_per_genre_target` for the current genre.
        # This is an estimate; actual calls might be fewer if targets are met or no more tracks are available.
        num_api_calls_for_genre_estimate = (num_tracks_per_genre_target + limit_per_api_call - 1) // limit_per_api_call

        for call_num in range(num_api_calls_for_genre_estimate):
            # Check overall and per-genre targets before making an API call
            if len(track_ids_seen) >= total_tracks_overall_target:
                break 
            if current_genre_track_count >= num_tracks_per_genre_target:
                print(f"  Target of {current_genre_track_count}/{num_tracks_per_genre_target} tracks reached for genre '{genre_item}'.")
                break

            try:
                # Fetch track recommendations based on the current seed genre.
                # The `recommendations` endpoint can take various parameters to tune results (e.g., target_*)
                # For simplicity, we're using only `seed_genres` and `limit`.
                print(f"  Making API call {call_num + 1}/{num_api_calls_for_genre_estimate} for genre '{genre_item}'...")
                results = sp.recommendations(seed_genres=[genre_item], limit=limit_per_api_call)
                
                if not results or not results['tracks']:
                    print(f"  ⚠️ No tracks returned by API for genre '{genre_item}' in this call. Recommendations might be exhausted for this query.")
                    break # Stop trying for this genre if Spotify returns no tracks for this specific recommendation query

                # Extract track IDs from the current batch of recommendations
                current_batch_track_ids = [track['id'] for track in results['tracks'] if track and track.get('id')]
                
                # Filter out tracks already seen globally or those without valid IDs from this batch
                new_unique_track_ids_in_batch = [tid for tid in current_batch_track_ids if tid and tid not in track_ids_seen]

                if not new_unique_track_ids_in_batch:
                    # print(f"  No new unique tracks in this batch for genre '{genre_item}'.")
                    continue # Move to the next API call (if any) or next genre if no new tracks found in this batch

                # Collect detailed information for the new unique tracks found in this batch
                batch_track_details_list = []
                for track_data_from_api in results['tracks']:
                    track_id = track_data_from_api.get('id')
                    # Ensure this track ID is one of the new unique ones identified for this batch
                    if track_id and track_id in new_unique_track_ids_in_batch:
                        # Double-check per-genre target before adding detailed info
                        if current_genre_track_count >= num_tracks_per_genre_target: break 

                        track_info_dict = {
                            'id': track_id,
                            'name': track_data_from_api.get('name'),
                            'popularity': track_data_from_api.get('popularity'),
                            'artist_name': track_data_from_api['artists'][0]['name'] if track_data_from_api.get('artists') else None,
                            'artist_id': track_data_from_api['artists'][0]['id'] if track_data_from_api.get('artists') else None,
                            'album_name': track_data_from_api.get('album', {}).get('name'), # Safer access for nested dict
                            'album_id': track_data_from_api.get('album', {}).get('id'),   # Safer access
                            'preview_url': track_data_from_api.get('preview_url'),
                            'external_url': track_data_from_api.get('external_urls', {}).get('spotify'), # Safer access
                            'genre_seed': genre_item, # Store the seed genre for context/later analysis
                            'explicit': track_data_from_api.get('explicit', False) # Get 'explicit' status, default to False
                        }
                        batch_track_details_list.append(track_info_dict)
                        track_ids_seen.add(track_id) # Add to master set of seen track IDs
                        current_genre_track_count += 1
                
                # Fetch audio features for the newly collected tracks in this batch
                if batch_track_details_list:
                    ids_for_audio_features_query = [t['id'] for t in batch_track_details_list]
                    audio_features_api_results = sp.audio_features(tracks=ids_for_audio_features_query)
                    
                    # Create a dictionary for efficient lookup of audio features by track ID
                    features_lookup_dict = {f['id']: f for f in audio_features_api_results if f} # Ensure f is not None
                    
                    # Merge audio features into their respective track detail dictionaries
                    for track_item_dict in batch_track_details_list:
                        features = features_lookup_dict.get(track_item_dict['id'])
                        if features: # Check if audio features were found for this track
                            track_item_dict.update({
                                'danceability': features.get('danceability'), 'energy': features.get('energy'),
                                'key': features.get('key'), 'loudness': features.get('loudness'),
                                'mode': features.get('mode'), 'speechiness': features.get('speechiness'),
                                'acousticness': features.get('acousticness'), 
                                'instrumentalness': features.get('instrumentalness'),
                                'liveness': features.get('liveness'), 'valence': features.get('valence'),
                                'tempo': features.get('tempo'), 'duration_ms': features.get('duration_ms'),
                                'time_signature': features.get('time_signature')
                            })
                        # Add the enriched track data (metadata + audio features) to the main list for DataFrame creation
                        all_tracks_data.append(track_item_dict) 
                
                print(f"  Fetched {current_genre_track_count} tracks for '{genre_item}'. Total unique tracks collected: {len(track_ids_seen)}.")
                
                # Be respectful to the API: A short pause between API calls.
                time.sleep(0.6) # Pause for 600 milliseconds

            except spotipy.SpotifyException as e:
                print(f"  ❌ Spotify API Error during recommendations for genre '{genre_item}': HTTP {e.http_status} - {e.msg}")
                if e.http_status == 429: # Rate limit error
                    # Spotify API often provides a 'Retry-After' header indicating how many seconds to wait.
                    retry_after_seconds = int(e.headers.get('Retry-After', 30)) # Default to 30s if header not found
                    print(f"  🕒 Rate limit hit. Spotify API advises waiting for {retry_after_seconds} seconds...")
                    time.sleep(retry_after_seconds)
                else: # Other Spotify API errors (e.g., 401 Unauthorized, 403 Forbidden, 500 Internal Server Error)
                    print(f"  Skipping to next API call or genre due to unrecoverable Spotify API error: HTTP {e.http_status}.")
                    time.sleep(5) # Brief general pause for other errors before continuing
            except Exception as e: # Catch any other unexpected errors during processing for a genre
                print(f"  ❌ An unexpected error occurred while processing genre '{genre_item}': {type(e).__name__} - {str(e)}")
                print(f"     Problematic track data might be skipped. Continuing with next batch or genre if possible.")
                time.sleep(5) # Brief pause
    
    # After iterating through all genres or meeting the track target
    if not all_tracks_data:
        print("\n🔴 No track data was collected successfully. This could be due to persistent API issues,")
        print("   incorrect API configuration, or issues with the provided genre seeds yielding no results.")
        return None

    # Convert the list of track data dictionaries into a pandas DataFrame
    tracks_df = pd.DataFrame(all_tracks_data)
    print(f"\n📊 Total tracks collected and dictionary entries created before final processing: {len(tracks_df)}")
    
    # Final de-duplication based on track ID (this should be largely redundant if track_ids_seen logic is perfect, but acts as a good safeguard)
    tracks_df.drop_duplicates(subset=['id'], keep='first', inplace=True)
    print(f"   Unique tracks after final de-duplication based on 'id': {len(tracks_df)}")
    
    # Drop rows where essential information might still be missing (e.g., if API returned partial data for some reason)
    # The .get() method used during data extraction should prevent most KeyErrors if fields are missing from API response,
    # resulting in NaN values instead, which can then be handled by dropna or later imputation.
    essential_fields_for_dropna = ['id', 'name', 'artist_name'] # Consider adding 'popularity' and core audio features if strictly needed and not imputable
    tracks_df.dropna(subset=essential_fields_for_dropna, inplace=True)
    print(f"   Tracks remaining after ensuring essential fields ({', '.join(essential_fields_for_dropna)}) are present: {len(tracks_df)}")
    
    # Final check on target achievement
    if len(tracks_df) < total_tracks_overall_target:
        print(f"\n⚠️ Warning: Collected {len(tracks_df)} unique and valid tracks, which is less than the target of {total_tracks_overall_target}.")
        print(f"   This could be due to API limitations, strictness of queries, or exhaustion of available tracks for given seeds.")
    else:
        print(f"\n✅ Successfully collected {len(tracks_df)} unique and valid tracks, meeting or exceeding the overall target.")
    
    return tracks_df

# Print a confirmation that the function is defined and ready.
print("✅ Spotify data fetching function `fetch_spotify_track_data` is defined and ready to be called.")
print("   This function includes type hints, detailed docstrings, and enhanced error handling.")

✅ Spotify data fetching function `fetch_spotify_track_data` is defined and ready to be called.
   This function includes type hints, detailed docstrings, and enhanced error handling.


In [4]:
# Cell 5: Execute Data Collection

print("🚀 Initiating Spotify Data Collection Process...")

# Ensure API credentials are valid before proceeding
# This check references the global client_id and client_secret from Cell 3
if not client_id or client_id == 'YOUR_CLIENT_ID' or \
   not client_secret or client_secret == 'YOUR_CLIENT_SECRET':
    print("="*80)
    print("❌ EXECUTION HALTED: Spotify API credentials are not correctly set in Cell 3.")
    print("   Please configure them either as environment variables or directly in Cell 3 for development.")
    print("   Cannot proceed with data collection.")
    print("="*80)
    songs_df_raw = None  # Ensure variable exists even if collection is skipped
else:
    # Call the data fetching function defined in Cell 4.
    print("   Calling the `fetch_spotify_track_data` function...")
    songs_df_raw = fetch_spotify_track_data( # <--- Corrected function name
        client_id_param=client_id,
        client_secret_param=client_secret,
        genres_list=genres,
        num_tracks_per_genre_target=num_tracks_per_genre,
        total_tracks_overall_target=total_tracks_target
    )

# --- Post-Collection Summary ---
if songs_df_raw is not None and not songs_df_raw.empty:
    print("\n🎉 Spotify Data Collection Process Finished Successfully!")
    print(f"   Collected a total of {len(songs_df_raw)} unique and valid tracks.")
    print(f"   The raw dataset has {songs_df_raw.shape[1]} features (columns).")
elif songs_df_raw is not None and songs_df_raw.empty:
    print("\n⚠️ Data collection process completed, but the resulting DataFrame is empty.")
    print("   Please review the logs from the function call above for API errors or issues with track fetching.")
else: # songs_df_raw is None
    print("\n❌ Data collection process failed or was halted due to critical errors (e.g., authentication).")
    print("   Please review the error messages from the function call above.")

HTTP Error for GET to https://api.spotify.com/v1/recommendations with Params: {'limit': 75, 'seed_genres': 'pop'} returned 404 due to None


🚀 Initiating Spotify Data Collection Process...
   Calling the `fetch_spotify_track_data` function...
✅ Successfully authenticated with Spotify API for data fetching.

🔄 Initializing data collection. Overall target: 5500 unique tracks.
   Will attempt to fetch up to 400 tracks per genre from 20 seed genres.

🎵 Processing Genre: 'pop' (Collected so far: 0 unique tracks)
  Making API call 1/6 for genre 'pop'...
  ❌ Spotify API Error during recommendations for genre 'pop': HTTP 404 - https://api.spotify.com/v1/recommendations?limit=75&seed_genres=pop:
 None
  Skipping to next API call or genre due to unrecoverable Spotify API error: HTTP 404.


HTTP Error for GET to https://api.spotify.com/v1/recommendations with Params: {'limit': 75, 'seed_genres': 'pop'} returned 404 due to None


  Making API call 2/6 for genre 'pop'...
  ❌ Spotify API Error during recommendations for genre 'pop': HTTP 404 - https://api.spotify.com/v1/recommendations?limit=75&seed_genres=pop:
 None
  Skipping to next API call or genre due to unrecoverable Spotify API error: HTTP 404.


HTTP Error for GET to https://api.spotify.com/v1/recommendations with Params: {'limit': 75, 'seed_genres': 'pop'} returned 404 due to None


...

HTTP Error for GET to https://api.spotify.com/v1/recommendations with Params: {'limit': 75, 'seed_genres': 'rock'} returned 404 due to None


  Making API call 5/6 for genre 'rock'...
  ❌ Spotify API Error during recommendations for genre 'rock': HTTP 404 - https://api.spotify.com/v1/recommendations?limit=75&seed_genres=rock:
 None
  Skipping to next API call or genre due to unrecoverable Spotify API error: HTTP 404.


HTTP Error for GET to https://api.spotify.com/v1/recommendations with Params: {'limit': 75, 'seed_genres': 'rock'} returned 404 due to None


  Making API call 6/6 for genre 'rock'...
  ❌ Spotify API Error during recommendations for genre 'rock': HTTP 404 - https://api.spotify.com/v1/recommendations?limit=75&seed_genres=rock:
 None
  Skipping to next API call or genre due to unrecoverable Spotify API error: HTTP 404.


KeyboardInterrupt: 